In [2]:
import json
import re

def clean_data(data):
    cleaned_data = []

    for university in data:
        cleaned_university = {
            "มหาวิทยาลัย": university["มหาวิทยาลัย"],
            "รายละเอียด": {}
        }

        for branch, programs in university["รายละเอียด"].items():
            clean_branch = re.sub(r"^\d+\.\s*", "", branch)
            cleaned_university["รายละเอียด"][clean_branch] = []

            for program in programs:
                clean_program = {}
                for key, value in program.items():
                    if key == "":
                        continue
                    elif key.startswith("รอบ"):
                        if "ไม่เปิดรับสมัครในรอบนี้" in value:
                            clean_program[key] = 0
                        else:
                            match = re.search(r"\d+", value)
                            clean_program[key] = int(match.group()) if match else 0
                    else:
                        clean_program[key] = value

                clean_program = {k: v for k, v in clean_program.items() if v != ""}
                cleaned_university["รายละเอียด"][clean_branch].append(clean_program)

        cleaned_data.append(cleaned_university)

    return cleaned_data

with open("../data/all_universities_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

cleaned_data = clean_data(data)

with open("../data/cleaned_universities_data.json", "w", encoding="utf-8") as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

print("Data cleaned and saved to cleaned_data.json")


Data cleaned and saved to cleaned_data.json


#Convert to CSV

In [11]:
import json
import csv

# Load JSON data
with open("../data/cleaned_universities_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Function to replace empty values or "-" with "ไม่ระบุ"
def replace_empty_values(value):
    if value == "" or value == "-" or value == "_":
        return "ไม่ระบุ"
    return value

# Prepare CSV data
csv_data = []
csv_headers = [
    "มหาวิทยาลัย", "คณะ", "ชื่อหลักสูตร", "ชื่อหลักสูตรภาษาอังกฤษ", "ประเภทหลักสูตร", "วิทยาเขต",
    "ค่าใช้จ่าย", "อัตราการสำเร็จการศึกษา", "อัตราการได้งานทำ", "ค่ามัธยฐานเงินเดือน",
    "รอบ 1 Portfolio", "รอบ 2 Quota", "รอบ 3 Admission", "รอบ 4 Direct Admission"
]

for university in data:
    university_name = university["มหาวิทยาลัย"]
    for branch, programs in university["รายละเอียด"].items():
        for program in programs:
            row = {
                "มหาวิทยาลัย": replace_empty_values(university_name),
                "คณะ": replace_empty_values(branch),
                "ชื่อหลักสูตร": replace_empty_values(program.get("ชื่อหลักสูตร", "")),
                "ชื่อหลักสูตรภาษาอังกฤษ": replace_empty_values(program.get("ชื่อหลักสูตรภาษาอังกฤษ", "")),
                "ประเภทหลักสูตร": replace_empty_values(program.get("ประเภทหลักสูตร", "")),
                "วิทยาเขต": replace_empty_values(program.get("วิทยาเขต", "")),
                "ค่าใช้จ่าย": replace_empty_values(program.get("ค่าใช้จ่าย", "")),
                "อัตราการสำเร็จการศึกษา": replace_empty_values(program.get("อัตราการสำเร็จการศึกษา", "")),
                "อัตราการได้งานทำ": replace_empty_values(program.get("อัตราการได้งานทำ", "")),
                "ค่ามัธยฐานเงินเดือน": replace_empty_values(program.get("ค่ามัธยฐานเงินเดือน", "")),
                "รอบ 1 Portfolio": replace_empty_values(program.get("รอบ 1 Portfolio", "")),
                "รอบ 2 Quota": replace_empty_values(program.get("รอบ 2 Quota", "")),
                "รอบ 3 Admission": replace_empty_values(program.get("รอบ 3 Admission", "")),
                "รอบ 4 Direct Admission": replace_empty_values(program.get("รอบ 4 Direct Admission", ""))
            }
            csv_data.append(row)

# Save CSV data
with open("../data/cleaned_universities_data.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=csv_headers)
    writer.writeheader()
    writer.writerows(csv_data)

print("Data has been converted to CSV and saved to data.csv")


Data has been converted to CSV and saved to data.csv
